In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from astropy.table import Table, Column

In [2]:
import piscola
import sncosmo
import sfdmap

In [3]:
import psycopg2
import requests
from sqlalchemy import create_engine

In [4]:
url = "https://raw.githubusercontent.com/alercebroker/usecases/master/alercereaduser_v4.json"
params = requests.get(url).json()['params']
conn = psycopg2.connect(dbname=params['dbname'], user=params['user'], host=params['host'], password=params['password'])

# MW extinction

Location of the Milky Way dust map of SFD (Schleger, Finkbeiner & Davis 1998), downloaded from: https://github.com/kbarbary/sfdmap

In [5]:
# directory of the dustmap
dustmap_dir = '/home/francisca/sfddata-master/'

with piscola:

```scaling = 1.``` original dustmap of SFD, to use in SALT2

```scaling = 0.86``` or ```None``` correction of Schlafly & Finkbeiner 2011, to use in SALT3

```piscola.extinction_correction.calculate_ebv(ra, dec, scaling=0.86, dustmaps_dir=dustmap_dir)```

# Bandpasses

ZTF bandpasses with no atmospheric correction: https://sncosmo.readthedocs.io/en/latest/bandpass-list.html

In [6]:
ztfg = sncosmo.get_bandpass('ztfg')
ztfr = sncosmo.get_bandpass('ztfr')

wavelengths of the filters:

```ztfg.wave```

```ztfr.wave```

transmissions of the filters:
    
```ztfg.trans```

```ztfr.trans```

effective (transmission-weighted) wavelength of the filters:
    
```ztfg.wave_eff```

```ztfr.wave_eff```

# Primeras SNIa confirmadas espectroscópicamente de ZTF

## Obtener metadata y detecciones, y agregar el redshift espectroscópico en metadata

Lista de 121 SNIa confirmadas espectroscópicamente de ZTF (https://iopscience.iop.org/article/10.3847/1538-4357/ab4cf5/pdf)
su rango de redshift es $0.0181 \leq z \leq 0.141$.

In [7]:
early_obs =  ('ZTF18aasdted', 'ZTF18abgmcmv', 'ZTF18abauprj', 
              'ZTF18aaxcntm', 'ZTF18abcflnz', 'ZTF18abcrxoj', 
              'ZTF18abuqugw', 'ZTF18aaxsioa', 'ZTF18abkhcwl', 
              'ZTF18abfhryc', 'ZTF18aaxdrjn', 'ZTF18aaumeys', 
              'ZTF18abkhcrj', 'ZTF18abeecwe', 'ZTF18aawjywv', 
              'ZTF18abpaywm', 'ZTF18aayjvve', 'ZTF18aazsabq', 
              'ZTF18aaslhxt', 'ZTF18abbvsiv', 'ZTF18aawurud',
              'ZTF18abwdcdv', 'ZTF18abucvbf', 'ZTF18aapqwyv', 
              'ZTF18abjvhec', 'ZTF18aazixbw', 'ZTF18aansqun', 
              'ZTF18abcsgvj', 'ZTF18abrzeym', 'ZTF18abdbuty', 
              'ZTF18aaqcugm', 'ZTF18absdgon', 'ZTF18aavrwhu', 
              'ZTF18abckujq', 'ZTF18aaqffyp', 'ZTF18abxxssh', 
              'ZTF18sbsxlpi', 'ZTF18abwmuua', 'ZTF18abetehf', 
              'ZTF18abssuxz', 'ZTF18abmmkaz', 'ZTF18aazblzy',
              'ZTF18abnvoel', 'ZTF18abealop', 'ZTF18abbpeqo', 
              'ZTF18abcysdx', 'ZTF18aaqnrum', 'ZTF18abfgygp', 
              'ZTF18abdefet', 'ZTF18abdfwur', 'ZTF18aavrzxp', 
              'ZTF18abeegsl', 'ZTF18abmxdhb', 'ZTF18aazabmh', 
              'ZTF18aaunfqq', 'ZTF18aaqcqvr', 'ZTF18aapsedq', 
              'ZTF18abpamut', 'ZTF18aazjztm', 'ZTF18aaqcozd', 
              'ZTF18abckujg', 'ZTF18aatzygk', 'ZTF18abjtgdo', 
              'ZTF18abetewu', 'ZTF18aaytovs', 'ZTF18abjstcm', 
              'ZTF18abokpvh', 'ZTF18abpmmpo', 'ZTF18aailmnv', 
              'ZTF18abdkimx', 'ZTF18aaqqoqs', 'ZTF18aaxwjmp', 
              'ZTF18aaydmkh', 'ZTF18abcecfi', 'ZTF18abxygvv', 
              'ZTF18abdfydj', 'ZTF18abdfazk', 'ZTF18abdmgab', 
              'ZTF18aauhxce', 'ZTF18abqjvyl', 'ZTF18abimsyv', 
              'ZTF18aazcoob', 'ZTF18aasesgl', 'ZTF18abpttky',
              'ZTF18aaumlfl', 'ZTF18abkifng', 'ZTF18abtnlik', 
              'ZTF18abfhaji', 'ZTF18abfzkno', 'ZTF18aaxvpsw', 
              'ZTF18abkdujo', 'ZTF18abkigee', 'ZTF18aaoxryq', 
              'ZTF18abclalx', 'ZTF18abwnsoc', 'ZTF18abssdpi', 
              'ZTF18aauocnw', 'ZTF18aaxqyki', 'ZTF18abukmty', 
              'ZTF18abwtops', 'ZTF18abfwuwn', 'ZTF18abkhdxe', 
              'ZTF18abtogdl', 'ZTF18abgxvra', 'ZTF18abjtger',
              'ZTF18aarldnh', 'ZTF18aaxrvzj', 'ZTF18aboaeqy', 
              'ZTF18aarqnje', 'ZTF18aaqcqkv', 'ZTF18abptsco', 
              'ZTF18abrzrnb', 'ZTF18aaxakhh', 'ZTF18abixjey', 
              'ZTF18abslxhz', 'ZTF18abvbayb', 'ZTF18abqbavl', 
              'ZTF18abtcdfv', 'ZTF18abjdjge', 'ZTF18abatffv', 
              'ZTF18abklljv')

lista_early_obs = list(early_obs)
array_early_obs = np.array(lista_early_obs)

Redshift de las 121 SNIa confirmadas espectroscópicamente de ZTF

In [8]:
early_red = [0.0181, 0.0185, 0.0242, 0.0269, 0.0273, 0.0309, 
             0.0313, 0.0315, 0.0317, 0.0323, 0.034, 0.0365, 0.0383,
             0.0393, 0.04, 0.045, 0.0474, 0.05, 0.0509, 0.051, 
             0.0531, 0.0538, 0.0549, 0.056, 0.057, 0.0594, 0.0597, 
             0.06, 0.06, 0.06, 0.0619, 0.062, 0.062, 0.0638, 0.064,
             0.064, 0.0642, 0.0643, 0.0649, 0.0649, 0.065, 0.0653, 
             0.065, 0.0666, 0.0667, 0.067, 0.069, 0.07, 0.07, 0.07,
             0.07, 0.07, 0.07, 0.07, 0.0711, 0.0716, 0.072, 0.072,
             0.0721, 0.0732, 0.074, 0.074, 0.0741, 0.074, 0.0746,
             0.079, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08,
             0.08, 0.08, 0.08, 0.0803, 0.0831, 0.0835, 0.084, 
             0.0845, 0.0859, 0.086, 0.0874, 0.088, 0.09, 0.09, 0.09,
             0.0916, 0.0921, 0.0936, 0.094, 0.1, 0.1, 0.1, 0.1, 0.1003,
             0.102, 0.104, 0.105, 0.105, 0.105, 0.106, 0.106, 0.1077, 
             0.110, 0.115, 0.115, 0.1174, 0.12, 0.12, 0.12, 0.1218, 
             0.13, 0.13, 0.1392, 0.14, 0.14, 0.14, 0.141]

lista_early_red = list(early_red)
array_early_red = np.array(lista_early_red)

Metadata de 121 SNIa confirmadas espectroscópicamente (incluyendo la clase con la que se clasificó en ALeRCE)

In [9]:
query_meta_early_SNIa = ''' 
         SELECT
           object.oid, object.meanra, object.meandec, object.ndet,
           object.firstMJD, object.deltajd, object.g_r_max,
           probability.classifier_name, probability.class_name,
           probability.ranking, probability.probability
         FROM
           object INNER JOIN probability
           ON object.oid=probability.oid
         WHERE 
          probability.classifier_name='lc_classifier'
          AND probability.ranking = 1
          AND object.oid IN ('ZTF18aasdted', 'ZTF18abgmcmv', 'ZTF18abauprj', 'ZTF18aaxcntm', 'ZTF18abcflnz', 'ZTF18abcrxoj', 'ZTF18abuqugw',
              'ZTF18aaxsioa', 'ZTF18abkhcwl', 'ZTF18abfhryc', 'ZTF18aaxdrjn', 'ZTF18aaumeys', 'ZTF18abkhcrj', 'ZTF18abeecwe',
              'ZTF18aawjywv', 'ZTF18abpaywm', 'ZTF18aayjvve', 'ZTF18aazsabq', 'ZTF18aaslhxt', 'ZTF18abbvsiv', 'ZTF18aawurud',
              'ZTF18abwdcdv', 'ZTF18abucvbf', 'ZTF18aapqwyv', 'ZTF18abjvhec', 'ZTF18aazixbw', 'ZTF18aansqun', 'ZTF18abcsgvj',
              'ZTF18abrzeym', 'ZTF18abdbuty', 'ZTF18aaqcugm', 'ZTF18absdgon', 'ZTF18aavrwhu', 'ZTF18abckujq', 'ZTF18aaqffyp',
              'ZTF18abxxssh', 'ZTF18sbsxlpi', 'ZTF18abwmuua', 'ZTF18abetehf', 'ZTF18abssuxz', 'ZTF18abmmkaz', 'ZTF18aazblzy',
              'ZTF18abnvoel', 'ZTF18abealop', 'ZTF18abbpeqo', 'ZTF18abcysdx', 'ZTF18aaqnrum', 'ZTF18abfgygp', 'ZTF18abdefet',
              'ZTF18abdfwur', 'ZTF18aavrzxp', 'ZTF18abeegsl', 'ZTF18abmxdhb', 'ZTF18aazabmh', 'ZTF18aaunfqq', 'ZTF18aaqcqvr',
              'ZTF18aapsedq', 'ZTF18abpamut', 'ZTF18aazjztm', 'ZTF18aaqcozd', 'ZTF18abckujg', 'ZTF18aatzygk', 'ZTF18abjtgdo', 
              'ZTF18abetewu', 'ZTF18aaytovs', 'ZTF18abjstcm', 'ZTF18abokpvh', 'ZTF18abpmmpo', 'ZTF18aailmnv', 'ZTF18abdkimx',
              'ZTF18aaqqoqs', 'ZTF18aaxwjmp', 'ZTF18aaydmkh', 'ZTF18abcecfi', 'ZTF18abxygvv', 'ZTF18abdfydj', 'ZTF18abdfazk',
              'ZTF18abdmgab', 'ZTF18aauhxce', 'ZTF18abqjvyl', 'ZTF18abimsyv', 'ZTF18aazcoob', 'ZTF18aasesgl', 'ZTF18abpttky',
              'ZTF18aaumlfl', 'ZTF18abkifng', 'ZTF18abtnlik', 'ZTF18abfhaji', 'ZTF18abfzkno', 'ZTF18aaxvpsw', 'ZTF18abkdujo',
              'ZTF18abkigee', 'ZTF18aaoxryq', 'ZTF18abclalx', 'ZTF18abwnsoc', 'ZTF18abssdpi', 'ZTF18aauocnw', 'ZTF18aaxqyki',
              'ZTF18abukmty', 'ZTF18abwtops', 'ZTF18abfwuwn', 'ZTF18abkhdxe', 'ZTF18abtogdl', 'ZTF18abgxvra', 'ZTF18abjtger',
              'ZTF18aarldnh', 'ZTF18aaxrvzj', 'ZTF18aboaeqy', 'ZTF18aarqnje', 'ZTF18aaqcqkv', 'ZTF18abptsco', 'ZTF18abrzrnb',
              'ZTF18aaxakhh', 'ZTF18abixjey', 'ZTF18abslxhz', 'ZTF18abvbayb', 'ZTF18abqbavl', 'ZTF18abtcdfv', 'ZTF18abjdjge',
              'ZTF18abatffv', 'ZTF18abklljv')
       '''
df_meta_early_SNIa = pd.read_sql_query(query_meta_early_SNIa, conn)

In [10]:
df_meta_early_SNIa

,oid,meanra,meandec,ndet,firstmjd,deltajd,g_r_max,classifier_name,class_name,ranking,probability
0,ZTF18aailmnv,214.202064,58.485409,33,58312.219097,56.913102,-0.263603,lc_classifier,SNIa,1,0.524000
1,ZTF18aansqun,251.292054,42.717867,15,58305.217025,40.962685,0.314913,lc_classifier,SNIbc,1,0.339720
2,ZTF18aaoxryq,248.843874,22.468449,13,58306.298056,29.937616,-0.190140,lc_classifier,SNIa,1,0.415840
3,ZTF18aaqcqvr,193.852766,45.578389,22,58242.247049,55.009873,0.406601,lc_classifier,SNIbc,1,0.466000
4,ZTF18aaqffyp,231.945012,34.948221,31,58244.331181,46.926181,-0.054800,lc_classifier,SNIbc,1,0.444216
...,...,...,...,...,...,...,...,...,...,...,...
81,ZTF18abucvbf,277.063688,46.353433,14,58370.159768,52.935185,-0.209349,lc_classifier,SNIa,1,0.402000
82,ZTF18abukmty,321.526517,24.045811,14,58370.274988,29.982072,-0.111900,lc_classifier,SNIa,1,0.389220
83,ZTF18abuqugw,244.557674,39.123812,18,58371.161875,151.346852,-0.220499,lc_classifier,SNIa,1,0.383688
84,ZTF18abvbayb,262.801136,79.054255,17,58373.264433,16.972199,-0.023264,lc_classifier,SNIa,1,0.369024


De las 121 SNIa se encontraron los metadata de 86 de estas SNIa

In [11]:
names_SNIa = np.array(df_meta_early_SNIa['oid'])

Seleccionar el redshift de aquellas SNIa de las cuales se encontró su metadata 

In [12]:
index_SNIa = np.in1d(array_early_obs, names_SNIa)
red_SNIa = array_early_red[index_SNIa]

Incluir el redshift en el metadata

In [13]:
df_z_early_SNIa = pd.DataFrame({'oid': lista_early_obs,
                                'z': lista_early_red})
df_meta_early_SNIa = df_meta_early_SNIa.set_index('oid').join(df_z_early_SNIa.set_index('oid'))
df_meta_early_SNIa = df_meta_early_SNIa.reset_index()
display(df_meta_early_SNIa)

,oid,meanra,meandec,ndet,firstmjd,deltajd,g_r_max,classifier_name,class_name,ranking,probability,z
0,ZTF18aailmnv,214.202064,58.485409,33,58312.219097,56.913102,-0.263603,lc_classifier,SNIa,1,0.524000,0.0800
1,ZTF18aansqun,251.292054,42.717867,15,58305.217025,40.962685,0.314913,lc_classifier,SNIbc,1,0.339720,0.0597
2,ZTF18aaoxryq,248.843874,22.468449,13,58306.298056,29.937616,-0.190140,lc_classifier,SNIa,1,0.415840,0.0940
3,ZTF18aaqcqvr,193.852766,45.578389,22,58242.247049,55.009873,0.406601,lc_classifier,SNIbc,1,0.466000,0.0716
4,ZTF18aaqffyp,231.945012,34.948221,31,58244.331181,46.926181,-0.054800,lc_classifier,SNIbc,1,0.444216,0.0640
...,...,...,...,...,...,...,...,...,...,...,...,...
81,ZTF18abucvbf,277.063688,46.353433,14,58370.159768,52.935185,-0.209349,lc_classifier,SNIa,1,0.402000,0.0549
82,ZTF18abukmty,321.526517,24.045811,14,58370.274988,29.982072,-0.111900,lc_classifier,SNIa,1,0.389220,0.1020
83,ZTF18abuqugw,244.557674,39.123812,18,58371.161875,151.346852,-0.220499,lc_classifier,SNIa,1,0.383688,0.0313
84,ZTF18abvbayb,262.801136,79.054255,17,58373.264433,16.972199,-0.023264,lc_classifier,SNIa,1,0.369024,0.1300


Incluir la extinción debido al polvo de la vía láctea en el metadata tanto para usar en ```SALT2 (mapa de polvo de SFD con la ley de extinción de O'Donnel 1994)```

y en ```SALT3 (corrección del mapa de polvo de SFD de Schlafly & Finkbeines (2011) con la ley de extinción de Fitzpatrick 1999)```.

In [14]:
def ext_corr_SALT2_g(ra, dec):
    A_g = piscola.extinction_correction.extinction_filter(ztfg.wave, ztfg.trans, 
                                                          ra, dec, scaling=1.0,
                                                          reddening_law='odonnell94',
                                                          dustmaps_dir=dustmap_dir, 
                                                          r_v=3.1)
    return A_g

def ext_corr_SALT2_r(ra, dec):
    A_r =piscola.extinction_correction.extinction_filter(ztfr.wave, ztfr.trans,
                                                          ra, dec, scaling=1.0,
                                                          reddening_law='odonnell94',
                                                          dustmaps_dir=dustmap_dir, 
                                                          r_v=3.1)
    return A_r

def ext_corr_SALT3_g(ra, dec):
    A_g = piscola.extinction_correction.extinction_filter(ztfg.wave, ztfg.trans, 
                                                          ra, dec, scaling=0.86,
                                                          reddening_law='fitzpatrick99',
                                                          dustmaps_dir=dustmap_dir, 
                                                          r_v=3.1)
    return A_g

def ext_corr_SALT3_r(ra, dec):
    A_r =piscola.extinction_correction.extinction_filter(ztfr.wave, ztfr.trans,
                                                         ra, dec, scaling=0.86,
                                                         reddening_law='fitzpatrick99',
                                                         dustmaps_dir=dustmap_dir, 
                                                         r_v=3.1)
    return A_r

In [15]:
df_meta_early_SNIa['A_g_SALT2'] = df_meta_early_SNIa.apply(lambda x: ext_corr_SALT2_g(x['meanra'], x['meandec']), axis=1)
df_meta_early_SNIa['A_r_SALT2'] = df_meta_early_SNIa.apply(lambda x: ext_corr_SALT2_r(x['meanra'], x['meandec']), axis=1)
df_meta_early_SNIa['A_g_SALT3'] = df_meta_early_SNIa.apply(lambda x: ext_corr_SALT3_g(x['meanra'], x['meandec']), axis=1)
df_meta_early_SNIa['A_r_SALT3'] = df_meta_early_SNIa.apply(lambda x: ext_corr_SALT3_r(x['meanra'], x['meandec']), axis=1)

Buscar las detecciones de las SNIa de las cuales se encontró su metadata

In [16]:
query_d_early_SNIa = ''' 
         SELECT
           oid, mjd, ra, dec, fid, magpsf, sigmapsf 
         FROM
           detection
         WHERE 
           oid IN ('ZTF18aailmnv', 'ZTF18aansqun', 'ZTF18aaoxryq', 'ZTF18aaqcqvr', 'ZTF18aaqffyp',
                   'ZTF18aaqqoqs', 'ZTF18aasdted', 'ZTF18aaslhxt', 'ZTF18aatzygk', 'ZTF18aaumeys',
                   'ZTF18aaumlfl', 'ZTF18aaunfqq', 'ZTF18aauocnw', 'ZTF18aavrwhu', 'ZTF18aavrzxp',
                   'ZTF18aawjywv', 'ZTF18aawurud', 'ZTF18aaxcntm', 'ZTF18aaxdrjn', 'ZTF18aaxqyki',
                   'ZTF18aaxrvzj', 'ZTF18aaxsioa', 'ZTF18aaxvpsw', 'ZTF18aaxwjmp', 'ZTF18aaydmkh',
                   'ZTF18aayjvve', 'ZTF18aaytovs', 'ZTF18aazblzy', 'ZTF18aazcoob', 'ZTF18aazixbw',
                   'ZTF18aazjztm', 'ZTF18aazsabq', 'ZTF18abauprj', 'ZTF18abbpeqo', 'ZTF18abbvsiv',
                   'ZTF18abcecfi', 'ZTF18abcflnz', 'ZTF18abckujg', 'ZTF18abclalx', 'ZTF18abcrxoj',
                   'ZTF18abcsgvj', 'ZTF18abdbuty', 'ZTF18abdefet', 'ZTF18abdfazk', 'ZTF18abdfwur',
                   'ZTF18abdkimx', 'ZTF18abealop', 'ZTF18abeecwe', 'ZTF18abetehf', 'ZTF18abetewu',
                   'ZTF18abfgygp', 'ZTF18abfhaji', 'ZTF18abfhryc', 'ZTF18abfzkno', 'ZTF18abgmcmv',
                   'ZTF18abimsyv', 'ZTF18abjtgdo', 'ZTF18abjtger', 'ZTF18abjvhec', 'ZTF18abkhcrj',
                   'ZTF18abkhcwl', 'ZTF18abkifng', 'ZTF18abkigee', 'ZTF18abmmkaz', 'ZTF18abmxdhb', 
                   'ZTF18abnvoel', 'ZTF18aboaeqy', 'ZTF18abokpvh', 'ZTF18abpamut', 'ZTF18abpaywm',
                   'ZTF18abpmmpo', 'ZTF18abptsco', 'ZTF18abpttky', 'ZTF18abqbavl', 'ZTF18abqjvyl',
                   'ZTF18abrzeym', 'ZTF18absdgon', 'ZTF18abssdpi', 'ZTF18abtcdfv', 'ZTF18abtnlik',
                   'ZTF18abtogdl', 'ZTF18abucvbf', 'ZTF18abukmty', 'ZTF18abuqugw', 'ZTF18abvbayb',
                   'ZTF18abxygvv')
         ORDER BY
           mjd
        '''
detect_early_SNIa = pd.read_sql_query(query_d_early_SNIa, conn)
len_unique_SNIa = len(detect_early_SNIa['oid'].unique())
print('cantidad de SNIa distintas en las detecciones', len_unique_SNIa)
display(detect_early_SNIa)

cantidad de SNIa distintas en las detecciones 86


,oid,mjd,ra,dec,fid,magpsf,sigmapsf
0,ZTF18aaqcqvr,58242.247049,193.852748,45.578390,1,20.141900,0.078784
1,ZTF18aaqffyp,58244.331181,231.944966,34.948222,2,19.233500,0.067199
2,ZTF18aaqffyp,58247.275718,231.944993,34.948245,2,18.807000,0.030836
3,ZTF18aaqffyp,58247.354444,231.945015,34.948217,1,18.728900,0.024548
4,ZTF18aaslhxt,58247.383634,276.108012,44.130393,1,20.714400,0.117904
...,...,...,...,...,...,...,...
2375,ZTF18aaumlfl,58617.286597,244.270727,50.792491,2,19.531965,0.199149
2376,ZTF18aaumlfl,58618.220012,244.270690,50.792461,2,19.607073,0.167359
2377,ZTF18aaxsioa,58618.228345,252.412938,45.492331,2,18.526710,0.133332
2378,ZTF18aaxsioa,58618.233009,252.412928,45.492305,2,18.603142,0.119479


In [17]:
detect_ZTF18aaqffyp = detect_early_SNIa[detect_early_SNIa.oid == 'ZTF18aaqffyp']
detect_ZTF18aaqffyp[detect_ZTF18aaqffyp.fid == 1]['magpsf']

3      18.728900
10     18.596500
21     18.376100
27     18.340100
50     18.530500
70     18.790200
158    19.678986
173    19.966927
184    19.831156
234    20.212010
389    20.690200
Name: magpsf, dtype: float64

### Separar según clase de supernova

In [18]:
SN_Ia = ['ZTF18aailmnv', 'ZTF18aansqun', 'ZTF18aaqcqvr', 'ZTF18aaoxryq',
         'ZTF18aaqffyp',                 'ZTF18aaslhxt', 'ZTF18aatzygk',
                         'ZTF18aaumlfl', 'ZTF18aaunfqq', 'ZTF18aauocnw',
         'ZTF18aavrwhu', 'ZTF18aavrzxp', 'ZTF18aawjywv', 'ZTF18aaxcntm',
         'ZTF18aawurud', 'ZTF18aaxdrjn', 'ZTF18aaxqyki', 'ZTF18aaxrvzj', 
         'ZTF18aaxsioa', 'ZTF18aaxvpsw', 'ZTF18aaxwjmp', 'ZTF18aaydmkh', 
         'ZTF18aayjvve',                 'ZTF18aazblzy', 'ZTF18aazcoob', 
         'ZTF18aazixbw', 'ZTF18aazjztm', 'ZTF18aazsabq',                                 
         'ZTF18abbvsiv', 'ZTF18abcecfi', 'ZTF18abcflnz', 'ZTF18abckujg',
         'ZTF18abclalx', 'ZTF18abcrxoj', 'ZTF18abcsgvj', 'ZTF18abdbuty',
         'ZTF18abdefet', 'ZTF18abdfazk', 'ZTF18abdfwur', 'ZTF18abdkimx',
         'ZTF18abealop', 'ZTF18abeecwe', 'ZTF18abetehf', 'ZTF18abetewu',
         'ZTF18abfgygp', 'ZTF18abfhaji', 'ZTF18abfhryc', 'ZTF18abfzkno',
                         'ZTF18abimsyv', 'ZTF18abjtgdo', 'ZTF18abjtger',
         'ZTF18abjvhec', 'ZTF18abkhcrj', 'ZTF18abkhcwl', 'ZTF18abkifng',
         'ZTF18abkigee', 'ZTF18abmmkaz', 'ZTF18abmxdhb', 'ZTF18abnvoel',
         'ZTF18aboaeqy', 'ZTF18abokpvh', 'ZTF18abpamut', 'ZTF18abpaywm',
         'ZTF18abpmmpo',                 'ZTF18abpttky', 'ZTF18abqbavl',
         'ZTF18abqjvyl', 'ZTF18abrzeym', 'ZTF18absdgon', 'ZTF18abssdpi',
         'ZTF18abtcdfv', 'ZTF18abtnlik', 'ZTF18abtogdl', 'ZTF18abucvbf',
         'ZTF18abukmty', 'ZTF18abuqugw', 'ZTF18abvbayb', 'ZTF18abxygvv']

SN_Ia_91T = ['ZTF18aaqqoqs', 'ZTF18aaumeys', 'ZTF18aaytovs', 'ZTF18abauprj',
             'ZTF18abgmcmv']

Hay algunas en las cuales no se indica su tipo (pero es entre Ia y peculiares)

In [19]:
Ia_tipo_no_especif = ['ZTF18aasdted', 'ZTF18abbpeqo', 'ZTF18abptsco']

Sus links en ALeRCE ZTF explorer son:
- https://alerce.online/object/ZTF18aasdted
- https://alerce.online/object/ZTF18abbpeqo
- https://alerce.online/object/ZTF18abptsco

# Funciones

In [20]:
def flujo(mag_AB, zp=23.9):
    '''
    Transforma magnitudes AB en flujo, por defecto el flujo se 
    transforma en unidades de microJanskys.
    '''
    flux = 10**((zp - mag_AB)/2.5)
    return flux

Para los ajustes es necesario tener al menos las siguientes columnas en los datos: tiempo, filtro, flujo, error del flujo, zp,
y sistema de magnitudes a la cual permite pasar el zp.
Por lo tanto, como los flujos de ZTF se entregan en magnitudes AB:
- sistema de magnitudes: AB.

- zp: 23.9 (permite pasar de flujo en $\mu J$ $\rightarrow$ magnitudes AB).

- se utilizan los flujos en microJanskys y sus errores calculados a partir de magnitudes AB y sus errores reportados por ZTF, como:
$$flux_{\mu J} = 10^{\frac{23.9 - mag_{AB}}{2.5}}$$
  con $mag_{AB}$ obtenida en la columna "magpsf" de las detecciones, y
$$fluxerr_{\mu J} = flux_{\mu J} \cdot magerror_{AB} \cdot \frac{\ln{10}}{2.5}$$
  con $magerror_{AB}$ obtenido en la columna "sigmapsf" de las detecciones.

## Versiones corregidas, hay que comparar las tablas con los flujos entre la correccion manual de la MW y la corrección con SDF (https://github.com/kbarbary/sfdmap)

(```scaling=0.86 para la corrección de Schlafly```)

In [21]:
def df_corrections_sncosmo(df_detections_oid, modelo, mw_ext_manual=True, zp=23.9, 
                           zpsys='ab'):
    '''
    A un dataframe de las detecciones de un objeto
    se le agregan nuevas columnas, necesarias para realizar los 
    ajustes, estas son: el flujo; el error del flujo; nombres de 
    filtros (tal como están guardados en sncosmo); el valor del zp 
    que permite pasar de magnitudes a flujo; y el sistema de 
    magnitudes al que permite pasar el zp.
    
    - el flujo se determina a partir de la función flux a partir 
    del valor de la magnitud AB en la columna "magpsf" de las 
    detecciones en el caso de que mw_ext_manual = False. En el caso 
    de que se aplique la extinción por el polvo de la vía láctea
    (mw_ext_manual = True), el valor de la extinción en el promedio 
    de las coordenadas en el que se ubica el objeto debe estar en los
    metadata (columna f).
    
    - el error del flujo se determina a partir del flujo y el error
    de las magnitudes como fluxerror = flux * magerror * ln(10)/2.5,
    donde este último factor sale de la transformación de 
    magnitudes a flujo, y magerr se encuentra en la columna 
    "sigmapsf" de las detecciones (sin hacer diferencias en si se 
    corrige manual o no la extinción de la vía láctea en la magnitud)
    (columna fe).
    
    - cambia los filtros para cada detección como fid: 1 -> 'ztfg' 
    y fid: 2 -> 'ztfr' (columna flt).
    
    - especifica el zp que permite pasar de magnitudes a flujo 
    (columna zp).
    
    - especifica el sistema de magnitudes al que permite pasar el 
    zeropoint (zpsys).
    
    Input:
    =================
    df_detections_oid: [pd.DataFrame], DataFrame que incluye las
                      detecciones un objeto.
    
    modelo: [str], nombre del modelo.
    
    mw_ext_manual: [bool], (default True) corregir manualmente por
                  la extinción de la vía láctea según el modelo 
                  utilizado
    
    zp: [float], zeropoint para pasar de flujo a magnitudes.
    
    zpsys: [str], sistema de magnitudes al que permite pasar el zeropoint.
    '''
    if mw_ext_manual:
        if modelo == 'salt2':
            mag_column = 'magpsf_SALT2'
        
        if modelo == 'salt3':
            mag_column = 'magpsf_SALT3'
    
    else:
        mag_column = 'magpsf'
    
    # transformar magnitudes a flujo
    df_detections_oid['f'] = df_detections_oid.apply(lambda x: flujo(x[mag_column]), axis=1)
    df_detections_oid['fe'] = df_detections_oid['sigmapsf'] *  df_detections_oid['f'] * (np.log(10)/2.5)
    
    # cambiar fid a bandas de sncosmo
    band_dict = {1: 'ztfg', 2: 'ztfr'}
    df_detections_oid['band'] = df_detections_oid['fid'].map(band_dict)
    
    # indicar el sistema de magnitudes y el zp
    df_detections_oid['zp'] = zp
    df_detections_oid['zpsys'] = zpsys

In [22]:
def correct_to_fit_sncosmo(df_metadata, df_detections, modelo, mw_ext_manual=True, zp=23.9, zpsys='ab'):
    '''
    A partir de dataframes de los metadata de las SN y de sus
    detecciones construye una lista de tablas de astropy, que 
    es el formato de los datos que necesita sncosmo para hacer 
    los ajustes. En los dataframes de las detecciones, se incluyen
    columnas necesarias que se agregan con la función auxiliar 
    'df_corrections_sncosmo'.  
    
    Input:
    =================
    df_metadata: [pd.DataFrame], DataFrame que incluye los
                metadata de los objetos.
                  
    df_detections: [pd.DataFrame], DataFrame que incluye las
                  detecciones de los objetos.
    
    modelo: [str], nombre del modelo.
    
    mw_ext_manual: [bool], (default True) corregir manualmente por
                  la extinción de la vía láctea según el modelo 
                  utilizado
    
    zp: [float], zeropoint para pasar de flujo a magnitudes.
    
    zpsys: [str], sistema de magnitudes al que permite pasar el zeropoint.


    Output:
    =================
    table_list: [list[astropy.Table]], lista de tablas de astropy en el 
               formato apropiado para hacer los ajustes con sncosmo
    '''
    
    # habría que diferenciar la columna del flujo que se usa para los ajustes dependiendo de si se hace con 
    # SALT2 o SALT3 en la parte de mag_to_flux_flt
    oids = list(df_metadata.oid)
    
    table_list = []
    
    # iteración para corregir cada objeto
    for i in range(len(oids)):
        # oid de la SN
        oid = oids[i]
        # detecciones de la SN
        detect_i = df_detections[df_detections.oid == str(oid)]
        # metadata
        metadata_i = df_metadata[df_metadata.oid == str(oid)] 
        
        if mw_ext_manual:
        # según el modelo, aplicar la extinción de la MW (usando el mapa y 
        # ley de extinción con que se entrenó el modelo)
            detect_i_g = detect_i[detect_i.fid == 1]
            detect_i_r = detect_i[detect_i.fid == 2]
            if modelo == 'salt2':
                A_g = list(metadata_i['A_g_SALT2'])[0]
                A_r = list(metadata_i['A_r_SALT2'])[0]
                 
                # FUNCIONA PERO ENTREGA ERROR
                #detect_i.loc[detect_i.fid == 1, 'magpsf_SALT2'] = detect_i_g['magpsf'] - A_g
                #detect_i.loc[detect_i.fid == 2, 'magpsf_SALT2'] = detect_i_r['magpsf'] - A_r
                
                # Mismo error de antes
                #detect_i.loc[detect_i.fid == 1, 'magpsf_SALT2'] = detect_i.loc[detect_i.fid == 1, 'magpsf'] - A_g
                #detect_i.loc[detect_i.fid == 2, 'magpsf_SALT2'] = detect_i.loc[detect_i.fid == 2, 'magpsf'] - A_r
                
                detect_i['magpsf_SALT2'] = detect_i.apply(lambda x: x.magpsf - A_g if x.fid==1 else x.magpsf - A_r,
                                                         axis=1)
                
            if modelo == 'salt3':
                A_g = list(metadata_i['A_g_SALT3'])[0]
                A_r = list(metadata_i['A_r_SALT3'])[0]
                
                # FUNCIONA PERO ENTREGA ERROR
                #detect_i.loc[detect_i.fid == 1, 'magpsf_SALT3'] = detect_i_g['magpsf'] - A_g
                #detect_i.loc[detect_i.fid == 2, 'magpsf_SALT3'] = detect_i_r['magpsf'] - A_r
                
                # Mismo error de antes
                #detect_i.loc[detect_i.fid == 1, 'magpsf_SALT3'] = detect_i.loc[detect_i.fid == 1, 'magpsf'] - A_g
                #detect_i.loc[detect_i.fid == 2, 'magpsf_SALT3'] = detect_i.loc[detect_i.fid == 2, 'magpsf'] - A_r
                
                detect_i['magpsf_SALT3'] = detect_i.apply(lambda x: x.magpsf - A_g if x.fid==1 else x.magpsf - A_r,
                                                         axis=1)
            
        # aplicar las correcciones necesarias para hacer los ajustes con sncosmo
        df_corrections_sncosmo(detect_i, modelo, mw_ext_manual=mw_ext_manual, 
                               zp=zp, zpsys=zpsys)
        
        table_i = Table.from_pandas(detect_i)
        
        table_i.meta = dict(zip(list(metadata_i.columns), 
                                list(metadata_i.values[0])))
        
        table_list.append(table_i)
        
    return table_list

Para los ajustes:

- Se minimiza de $\chi^{2}$ mediante el minimizador 'minuit'.
- Un número máximo de iteraciones de 10.000.
- Se utilizan las detecciones que cumplen con tener una señal a ruido mayor a 3 (por defecto sncosmo utiliza 5).
- Se usarán los siguientes límites (por defecto) para los parámetros ajustados:
  - $z$: (0.001, 0.6)
  - $x1$: (-4 4) o (-3, 3)
  - $c$: (-0.4, 0.4) o (-0.3, 0.3)
  

Donde los límites para los parámetros $x1$ y $c$ son aquellos utilizados en Scolnic & Kesssler (2016) y Betoule et al. (2014) respectivamente.

- Se aplica la extinción de la vía láctea al modelo (afectando el sistema de referencia del observador), utilizando el mapa de polvo SFD (Schleger, Finkbeines & Davis 1998) con el valor de RA y DEC promedio del objeto y la ley de extinción de O'Donnel de 1994 para determinar el enrojecimiento $E(B-V)$, con parámetro $R_{V} = \frac{A_{V}}{E(B-V)} = 3.1$ constante en ```SALT2```. En ```SALT3``` este se determina con la corrección del mapa de polvo de SFD de Schlafly & Finkbeines (2011) y con la ley de extinción de Fitzpatrick 1999. Estos se fijan al modelo para cada SN.
- Todos estos pueden modificarse. En particular se puede cambiar O'Donnel 1994 por Clayton Cardelli & Mathis 1989 o por Fitzpatrick 1999 (estos vienen implementados en sncosmo pero también se puede cambiar por otro).

Además:

- Se puede incluir la covarianza del modelo (por defecto es falso), si es verdadero itera hasta converger.
- Se pueden descartar rangos de tiempo y de longitud de onda.

Nota:

- En particular, los ajustes funcionan sin límites para $x1$ y $c$, pero no funcionan si no hay límites para $z$ ya que después de cierto valor de $z$ los rangos de longitudes de onda de los filtros de ZTF quedan fuera del rango de longitud de onda del modelo (para ZTF este se determina con la longitud de onda mínima de la banda $g$ y es $z=0.82$, más detalles en el notebook "SALT2 y SALT3"). Con esto, los ajustes pueden realizarse solo con una banda, lo cual no permite determinar bien $c$, y por lo tanto los ajustes no son buenos. 


- Debido a lo conversado en la práctica acerca de que lo más probable es que ZTF no pueda tener observaciones por sobre $z=0.6$ decidí usar este valor como límite superior para los ajustes.


## POR CORREGIR (debería haber otra columna en la que ya se corrige por la extinción de la MW según las formas de SALT2 y SALT3 y usar esas columnas para calcular el flujo, los ajustes, etc)

### Más adelante se debe modificar para que en todas en las que encuentre un valor de $z$ espectroscópico cercano utilice ese valor (ver notebook de SN) o un rango de valores cerca

In [23]:
def ajuste(oid, df_metadata, astropy_table, modelo, version=None, 
           mw_ext_manual=True, mw_ext_modelo=False, z_real=False, 
           graficos=True, zp=23.9, zpsys='ab', minsnr=3.0, 
           z_bounds = (0.001, 0.6), x1_bounds = (-3, 3), 
           c_bounds = (-0.3, 0.3)):
    '''
    Ajusta el modelo especificado a las observaciones del oid 
    indicado para observaciones que cumplen con tener una cierta
    señal a ruido. Incluye límites para los valores que pueden 
    tomar los parámetros ajustados y efectos que se le incluyen 
    al modelo. Como resultado entrega una lista con el nombre 
    del objeto ajustado, el modelo que mejor se ajusta y los 
    resultados del ajuste. Por defecto se grafica el ajuste 
    realizado. 
    
    
    Input:
    =================
    oid: [str], identificador del objeto.
    
    df_metadata: [pd.DataFrame], DataFrame que incluye metadata 
                del objeto.
                
    astropy_table: [astropy.Table], tabla que contiene las 
                detecciones del objeto.
                
    modelo: [str], nombre del modelo.
                
    version: [str], versión del modelo.
    
    mw_ext_manual: [bool], (default True) corregir manualmente 
                 (en los datos) la extinción de la vía láctea
                 si es False, se aplica en el ajuste como 
                 parámetro fijo de acuerdo al modelo utilizado.
                 
    mw_ext_modelo: [bool], (default False) aplicar la extinción
                 por el polvo de la vía láctea en el modelo, esta
                 se aplica en el ajuste como parámetro fijo con la
                 ubicación promedio del objeto y el mapa y ley de
                 extinción de acuerdo al modelo utilizado.
    
    z_real: [bool], (default False) incluir el redshift real en el 
           ajuste.
    
    graficos: [bool], (default True) graficar el ajuste y los datos.
    
    zp: [float], (default 23.9) zeropoint para pasar de flujo a 
                magnitudes (microjanskys a magnitudes AB).
    
    minsnr: [float], (default 3.0) mínima señal a ruido de los datos 
           para realizar el ajuste con ellos, en la función de ajuste
           de sncosmo este valor es 5.0 por defecto.
           
    z_bounds: [2-tuple of floats], (default (0.001, 0.6)) límites para el 
             redshift ajustado.
               
    x1_bounds: [2-tuple of floats], (default (-3, 3)) límites para el 
              parámetro x1 ajustado (usado por Betoule et al. (2014)).
               
    c_bounds: [2-tuple of floats], (default (-0.3, 0.3)) límites para    
             el parámetro c ajustado (usado por Betoule et al. (2014)).
    
    
    
    Output:
    =================
    
    entrega una lista con 3 elementos:
    1.- nombre del objeto ajustado
    2.- modelo ajustado
    3.- resultados del ajuste
    
    Por defecto grafica el ajuste.
    
    ----------------------------------------------------------------------
    Extras:
    =======
    en el caso en que no se corrige la extinción de la vía láctea
    manualmente se utilizan las funciones de sncosmo, que utilizan los 
    siguientes parámetros.
    
    effects: [str or list of str], efectos que se incluyen en el 
            modelo. 
            
            Por defecto en el ajuste con SALT2 se utiliza 
            el mapa de polvo de SFD (usando los valores de ra y dec 
            promedio del objeto) para la extinción de O'Donnel de 
            1994 ('OD94') para determinar el enrojecimiento E(B-V) 
            con parámetro R_V = A_V/E(B-V) = 3.1 constante.
            
            En el ajuste con SALT3 se utiliza la corrección del
            el mapa de polvo de SFD de Schlafly & Finkbeiner 2011 
            (usando los valores de ra y dec promedio del objeto) 
            con la ley de extinción de Fitzpactrick de 1999 ('F99')
            para determinar el enrojecimiento E(B-V) con parámetro 
            R_V = A_V/E(B-V) = 3.1 constante.
            
    effect_names: [str or list of str], prefijo que se le añade
                 a los parámetros del efecto. Por defecto en el
                 ajuste solo se incluye la extinción de la vía
                 láctea ('mw')
                 
    effect_frames: [str or list of str], sistemas de referncia
                  a los cuales afecta el efecto que se le agrega
                  al modelo
    ----------------------------------------------------------------------
    '''
    df_metadata = df_metadata.reset_index() 
    
    oid_table = []
    ind_table = []
    for i in range(len(astropy_table)):
        table_i = astropy_table[i]
        oid_i = list(table_i['oid'])[0]
        oid_table.append(oid_i)
        ind_table.append(i)
    array_oid_table = np.array(oid_table)
    
    mask = np.where(array_oid_table==str(oid))[0]
    list_index = np.intersect1d(mask, ind_table)
    index = int(list_index[0])
    table = astropy_table[index]
    
    # inicializar la fuente del modelo
    if version is not None:
        source = sncosmo.get_source(modelo, version=version)

    else:
        source = sncosmo.get_source(modelo)
    
    # aplicar los efectos de enrojecimiento por el polvo de la vía láctea
    # en el caso de que este no se haya corregido manualmente y se indique
    # que se quiere aplicar en el modelo
    if not mw_ext_manual and mw_ext_modelo:
        meanra = table.meta['meanra']
        meandec = table.meta['meandec']
        rv = 3.1
        
        if modelo == 'salt2':
            # effects = 'OD94'
            dust = sncosmo.OD94Dust()
            # nombre del efecto
            effect_name = 'mw'
            # sistema al que afecta
            effect_frame = 'obs'
            # agregar los efectos al modelo
            model = sncosmo.Model(source=source, effects=[dust], 
                                  effect_names=[effect_name], 
                                  effect_frames=[effect_frame])
            # dustmap of SFD
            m = sfdmap.SFDMap(dustmap_dir, scaling=1.0)
            # calcular E(B-V) con ese dustmap
            ebv = m.ebv(meanra, meandec)
            # fijarlos en el modelo
            model.set(mwebv=ebv, mwr_v=rv)
            
        if modelo == 'salt3':
            # effects = 'F99'
            dust = sncosmo.F99Dust()
            # nombre del efecto
            effect_name = 'mw'
            # sistema al que afecta
            effect_frame = 'obs'
            # agregar los efectos al modelo
            model = sncosmo.Model(source=source, effects=[dust], 
                                  effect_names=[effect_name], 
                                  effect_frames=[effect_frame])
            # dustmap of SFD
            m = sfdmap.SFDMap(dustmap_dir, scaling=0.86)
            # calcular E(B-V) con ese dustmap
            ebv = m.ebv(meanra, meandec)
            # fijarlos en el modelo
            model.set(mwebv=ebv, mwr_v=rv)

    else:
        model = sncosmo.Model(source=source)
        
    #if effect_names == 'host':
        #if effect_frames == 'rest':
            #model = sncosmo.Model(source=source, effects=[dust], 
            #effect_names=[effect_names], effect_frames=[effect_frames])
    
    if not z_real:
        bounds = {'z':z_bounds, 'x1':x1_bounds, 'c':c_bounds}
        result, fitted_model = sncosmo.fit_lc(table, model, 
                                              ['z', 't0', 'x0', 
                                               'x1', 'c'], # parametros a variar  
                                              bounds=bounds, 
                                              minsnr=minsnr) # elegir buenos bounds, 
                                                             # eso disminuye el tiempo de los ajustes
        
        if graficos:
            sncosmo.plot_lc(table, model=fitted_model, 
                            errors=result.errors, 
                            figtext= oid+' z ajustado '+modelo, 
                            zp=zp)
            print(" ==== Con ajuste del redshift ====")
            print('chisq/ndof: {}'.format(result['chisq']/result['ndof']))
        
    if z_real:
        z = table.meta['z']
        model.set(z=z)
        result, fitted_model = sncosmo.fit_lc(table, 
                                              model,
                                              ['t0', 'x0', 
                                               'x1', 'c'])
        if graficos:
            sncosmo.plot_lc(table, model=fitted_model, 
                            errors=result.errors,
                            figtext= oid+' z real '+modelo, 
                            zp=zp, minsnr=minsnr)
            print(" ===== Ajuste con redshift real ====")
            print('chisq/ndof: {}'.format(result['chisq']/result['ndof']))
    
    return [oid, fitted_model, result] 

In [24]:
def modelo_ajustado_y_resultados(df_metadata, df_detections, 
                                 modelo, version=None, 
                                 mw_ext_manual=True, mw_ext_modelo=False,
                                 z_real=False, zp=23.9, zpsys='ab',
                                 minsnr=3.0, z_bounds = (0.001, 0.6), 
                                 x1_bounds = (-3, 3), 
                                 c_bounds = (-0.3, 0.3),
                                 print_ind_and_oid=False):
    
    '''
    Ajusta los objetos cuyos nombres y metadata están en un 
    DataFrame (df_metadata) y cuyas detecciones están en otro
    DataFrame (df_detections). Utiliza las funciones auxiliares
    "correct_to_fit_sncosmo" para pasar las detecciones al 
    formato requerido; y "ajuste" para realizar el ajuste de cada 
    objeto (pero sin graficar los ajustes).
    
    Entrega una lista de los resultados de la función "ajuste" 
    de cada objeto, y, para poder encontrar objetos que no se 
    pudieron ajustar (en los cuales la función sncosmo.fit_lc 
    se cae) si se indica entre los parámetros 
    "print_ind_and_oid=True" imprime el valor del indice de las
    SN en metadata y el nombre de las SN que se han podido ajustar 
    (para poder identificar cuál SN no se puede ajustar, ya que
    imprimirá el oid de todas antes de que se caiga la función).
    
    
    Input:
    =================
    
    oid: [str], identificador del objeto.
    
    df_metadata: [pd.DataFrame], DataFrame que incluye metadata 
                del objeto.
                
    astropy_table: [astropy.Table], tabla que contiene las 
                detecciones del objeto.
                
    modelo: [str], nombre del modelo.
                
    version: [str], versión del modelo.
    
    mw_ext_manual: [bool], (default True) corregir manualmente 
                 (en los datos) la extinción de la vía láctea
                 si es False, se aplica en el ajuste como 
                 parámetro fijo de acuerdo al modelo utilizado.
                 
    mw_ext_modelo: [bool], (default False) aplicar la extinción
                 por el polvo de la vía láctea en el modelo, esta
                 se aplica en el ajuste como parámetro fijo con la
                 ubicación promedio del objeto y el mapa y ley de
                 extinción de acuerdo al modelo utilizado.
    
    z_real: [bool], (default False) incluir el redshift real en el 
           ajuste.
    
    zp: [float], (default 23.9) zeropoint para pasar de flujo a 
                magnitudes (microjanskys a magnitudes AB).
    
    minsnr: [float], (default 3.0) mínima señal a ruido de los datos 
           para realizar el ajuste con ellos, en la función de ajuste
           de sncosmo este valor es 5.0 por defecto.
           
    z_bounds: [2-tuple of floats], (default (0.001, 0.6)) límites para el 
             redshift ajustado.
               
    x1_bounds: [2-tuple of floats], (default (-3, 3)) límites para el 
              parámetro x1 ajustado (usado por Betoule et al. (2014)).
               
    c_bounds: [2-tuple of floats], (default (-0.3, 0.3)) límites para    
             el parámetro c ajustado (usado por Betoule et al. (2014)).
             
    print_ind_and_oid: [bool], (default False) imprimir el valor del índice 
                      y el oid de las SN que se han podido ajustar 
                      (secuencialmente de acuerdo a los oids de metadata)
    
    
    Output:
    =================
    
    Lista de los resultados de la función ajuste de cada SN, que es:
    
    lista con 3 elementos:
    1.- nombre del objeto ajustado
    2.- modelo ajustado
    3.- resultados del ajuste
    
    '''
    
    
    df_metadata = df_metadata.reset_index()
    df_detections = df_detections.reset_index()
    
    astropy_table = correct_to_fit_sncosmo(df_metadata, df_detections, 
                                           modelo, mw_ext_manual=mw_ext_manual, 
                                           zp=zp, zpsys=zpsys)
    lista_ajustes = []
    oids = list(df_metadata.oid)
    for i in range(len(astropy_table)):
        oid = oids[i]
        
        modelo_ajustado_y_resultados_i = ajuste(oid, df_metadata, 
                                                astropy_table,
                                                modelo, version=version,
                                                mw_ext_manual=mw_ext_manual,
                                                mw_ext_modelo=mw_ext_modelo,
                                                z_real=z_real, 
                                                graficos=False,
                                                zp=zp, zpsys=zpsys,
                                                minsnr=minsnr, 
                                                z_bounds=z_bounds, 
                                                x1_bounds=x1_bounds,
                                                c_bounds=c_bounds)

        lista_ajustes.append(modelo_ajustado_y_resultados_i)
        if print_ind_and_oid:
            print(i, oid)
    return lista_ajustes

## Creo que estos no necesitan modificación

Como se verá más adelante, un problema en el ajuste consiste en que los modelos SALT2 y SALT3 están definidos en un rango de tiempo con respecto al máximo de -20, 50 en el sistema de referencia en reposo, por lo que observaciones que se encuentran fuera de este rango aumentan mucho el valor de $\frac{\chi^{2}}{ndof}$ pese a que el ajuste se vea bien. 

Por la relación:
$$ z + 1 = \frac{\lambda_{obs}}{\lambda_{rest}}= \dfrac{\frac{c}{t_{obs}}}{\frac{c}{t_{rest}}}$$
con lo que 
$$ z + 1 = \frac{t_{rest}}{t_{obs}}$$

y así el rango de tiempo en $t_{obs}$ será: $-20 \cdot (1 + z)$, $50 \cdot (1 + z)$.

Por lo que una idea es determinar el tiempo del máximo ($t0$) y el redshift ($z$) en un primer ajuste y con estos hacer un segundo ajuste del modelo solo con las observaciones que se encuentren en el rango del modelo determinado con $t0$ y $z$.

Nota:
- Es importante notar que en el caso en que la selección de las observaciones que pertenecen al rango de tiempo del modelo se hace con el valor del redshift real ($z_{real}$) y los primeros ajustes realizados con este, se podría analizar qué pasaría si el ajuste determina el redshift real del modelo. Además asumiendo que en el primer ajuste se determina el valor del tiempo del máximo ($t_0$) de forma correcta o cercana a su valor real, se podría analizar qué pasaría en un "ajuste perfecto" en que se utilizan todos las observaciones que efectivamente pertenecen al rango de tiempo del modelo, y se ajustan solo los parámetros del modelo ($x_0$, $x1$, $c$) y el tiempo del máximo brillo ($t_0$). 

Pregunta:
- ¿Cómo se podrían incluir los errores del tiempo del máximo y del redshift determinados del primer ajuste en la determinación de este rango?

In [25]:
# tiempos máximos y mínimos en el sistema de referencia en reposo
# para los modelos SALT2 y SALT3
trest_min_modelo = -20 
trest_max_modelo = 50

def cortes_t_modelo(t_min, t_max, df_detect, df_results):
    '''
    A partir del tiempo máximo y mínimo del modelo en el 
    sistema de referencia en reposo, y los valores del tiempo
    del máximo (t0) y el redshift (z) determinados a partir
    de un primer ajuste, crea un nuevo DataFrame de las 
    detecciones de los objetos, que contiene solo las 
    observaciones que están dentro del rango de tiempo del modelo
    
    Input:
    ==============
    t_min: [float], tiempo mínimo del modelo con respecto al
           tiempo del máximo brillo, en el sistema de referencia
           en reposo
           
    t_max: [float], tiempo máximo del modelo con respecto al
           tiempo del máximo brillo, en el sistema de referencia
           en reposo
           
    df_detect: [pd.DataFrame], DataFrame que contiene las 
              detecciones de los objetos que se ajustaron en el
              primer ajuste
              
    df_results: [pd.DataFrame], DataFrame con los resultados del
               primer ajuste, obtenidos con la función 
               "estadisticas_resultados".
    
    
    Output:
    =============
    df_detect_cortes_t: [pd.DataFrame], DataFrame que contiene
                       las observaciones de los objetos dentro 
                       del rango de tiempo del modelo, determinado
                       a partir de los valores del tiempo del
                       máximo y redshift de un primer ajuste con 
                       el modelo.
    '''
    # lista de los oids que se ajustaron en el primer ajuste
    oids = list(df_results['oid'])
    
    # lista en la cual se guardarán las observaciones que están
    # en el rango de tiempo del modelo
    lista_df_detect_cortes_t = []
    
    # iterar para cada oid, guardando las observaciones que cumplen
    # lo anterior en la lista
    for i in range(len(oids)):
        oid_i = oids[i]
        # detecciones de cada oid
        detect_oid_i = df_detect[df_detect['oid'] == oid_i]
        ajustes_oid_i = df_results[df_results['oid'] == oid_i]
        
        # redshift ajustado
        z_i = list(ajustes_oid_i['z'])[0]
        
        # tiempo del máximo ajustado
        t0_i = list(ajustes_oid_i['t0'])[0]
        
        # cortes en tmax y tmin en el sistema de referencia del
        # observador, con tobs = trest*(1+z)
        corte_t_max = t0_i + t_max*(1 + z_i)
        corte_t_min = t0_i + t_min*(1 + z_i)
        
        # observaciones del oid en tiempos menores al máximo
        mask_t_max = detect_oid_i['mjd'] < corte_t_max
        df_detect_i_corte_t_max = detect_oid_i[mask_t_max]
        
        # observaciones del oid en el rango
        mask_rango_t = df_detect_i_corte_t_max['mjd'] > corte_t_min
        df_detect_i_cortes_t = df_detect_i_corte_t_max[mask_rango_t]
        
        # guardar las observaciones del oid que están en el rango
        # en una lista
        lista_df_detect_cortes_t.append(df_detect_i_cortes_t)
        
    # concatenar las observaciones de todos los oids que están
    # dentro del rango del modelo en un único DataFrame
    df_detect_cortes_t = pd.concat(lista_df_detect_cortes_t)
    
    return df_detect_cortes_t

Sin embargo, un problema de utilizar lo anterior es que se asume que tanto la fecha del máximo ajustada ($t_0$) como el redshift ($z$) se asume que son correctos, lo cual no siempre es cierto. En particular se verá que lo anterior hace que el valor de $\frac{\chi^{2}}{ndof}$ disminuya, pero sigue siendo alto, además aún se ven puntos que están fuera del rango del modelo, lo que puede pasar si el redshift del segundo ajuste es menor que el determinado en el primer ajuste. Algunas opciones para enfrentar lo anterior serían:
- Asumiendo que el valor del tiempo del máximo ajustado en el primer ajuste ($t_0$) es correcto, o cercano al valor correcto, descartar las observaciones fuera del rango del modelo en el sistema de referencia en reposo (en el cual $z=0$).

- Estimar de otra manera (más exacta) el tiempo del máximo y hacer lo mismo de la opción anterior con este tiempo del máximo.

- Determinar un rango de posibles valores de redshift de la SN a partir del valor de la magnitud aparente en el máximo, para que así el ajuste se demore menos y tome valores más probables (se verá que esto pasa más adelante usando límites para el valor del redshift).

Nota: 
- También se mostrará lo que pasa al eliminar puntos que pertenecen al rango de tiempo del modelo al analizar lo que pasa al hacer este corte en las detecciones para los ajustes que utilizan el redshift real, y así compararlo con el caso de un "ajuste perfecto" en que todos las observaciones seleccionadas para el ajuste pertenecen al rango de tiempo del modelo.

Preguntas:
- ¿Es posible determinar de forma más exacta el tiempo del máximo?

- ¿Sería posible determinar un rango de valores entre los cuales se puede encontrar el redshift de la SN? Usando la identificación de las galaxias anfitrionas (con CNN) y tal vez buscándo las que tienen galaxia anfitriona asignada (como en el notebook de SN).

In [26]:
def cortes_trest_modelo(t_min, t_max, df_detect, df_results):
    '''
    A partir del tiempo máximo y mínimo del modelo en el 
    sistema de referencia en reposo, y el valor del tiempo
    del máximo (t0) determinado a partir de un primer ajuste, 
    crea un nuevo DataFrame de las detecciones de los objetos, 
    que contiene solo las observaciones que están dentro del rango
    de tiempo de 20 días antes y 50 días después del máximo 
    determinado en el primer ajuste. Si t0 es correcto, entonces 
    ajustará las observaciones que siempre pertenecen al rango de 
    tiempo del modelo
    
    Input:
    ==============
    t_min: [float], tiempo mínimo del modelo con respecto al
           tiempo del máximo brillo, en el sistema de referencia
           en reposo
           
    t_max: [float], tiempo máximo del modelo con respecto al
           tiempo del máximo brillo, en el sistema de referencia
           en reposo
           
    df_detect: [pd.DataFrame], DataFrame que contiene las 
              detecciones de los objetos que se ajustaron en el
              primer ajuste
              
    df_results: [pd.DataFrame], DataFrame con los resultados del
               primer ajuste, obtenidos con la función 
               "estadisticas_resultados".
    
    
    Output:
    =============
    df_detect_cortes_t: [pd.DataFrame], DataFrame que contiene
                       las observaciones de los objetos dentro 
                       del rango de tiempo del modelo, como si
                       el objeto estuviese en el sistema de
                       referencia en reposo, determinado a partir 
                       del valor del tiempo del máximo determinado
                       de un primer ajuste con el modelo.
    '''
    # lista de los oids que se ajustaron en el primer ajuste
    oids = list(df_results['oid'])
    
    # lista en la cual se guardarán las observaciones que están
    # en el rango de tiempo del modelo, considerando que el 
    # objeto está en el sistema de referencia en reposo
    lista_df_detect_cortes_trest = []
    
    # iterar para cada oid, guardando las observaciones que cumplen
    # lo anterior en la lista
    for i in range(len(oids)):
        oid_i = oids[i]
        # detecciones de cada oid
        detect_oid_i = df_detect[df_detect['oid'] == oid_i]
        ajustes_oid_i = df_results[df_results['oid'] == oid_i]
        
        # tiempo del máximo ajustado
        t0_i = list(ajustes_oid_i['t0'])[0]
        
        # cortes en tmax y tmin
        corte_trest_max = t0_i + t_max
        corte_trest_min = t0_i + t_min
        
        # observaciones del oid en tiempos menores al máximo
        mask_trest_max = detect_oid_i['mjd'] < corte_trest_max
        df_detect_i_corte_trest_max = detect_oid_i[mask_trest_max]
        
        # observaciones del oid en el rango
        mask = df_detect_i_corte_trest_max['mjd'] > corte_trest_min
        df_detect_i_cortes_trest = df_detect_i_corte_trest_max[mask]
        
        # guardar las observaciones del oid que están en el rango
        # en una lista
        lista_df_detect_cortes_trest.append(df_detect_i_cortes_trest)
    
    # concatenar las observaciones de todos los oids que están
    # dentro del rango del modelo en un único DataFrame
    df_detect_cortes_trest = pd.concat(lista_df_detect_cortes_trest)
    
    return df_detect_cortes_trest

El objeto "result" de un ajuste (primer componente del resultado de la función sncosmo.fit_lc) es un diccionario con los siguientes parámetros:
- success: describe si el ajuste resultó.
- message: incluye mensajes adicionales de la salida.
- ncall: número de evaluaciones.
- chisq: mínimo valor de $\chi^{2}$.
- ndof: grados de libertad.
- param_names: nombre de los parámetros. 
- parameters: mejor ajuste de los parámetros.
- vparam_names: nombre de los parámetros que se variaron.
- covariance: arreglo 2-d de la covarianza de los parámetros (ordenados según vparam_names).
- errors: incerteza de los parámetros variados.
- nfit: número de veces que se realizó el ajuste, a menos de que se entregue el valor de la matriz de covarianza (donde itera hasta converger), su valor será siempre 1.
- data_mask: arreglo booleano que indica cuáles observaciones se utilizaron en el ajuste.

In [27]:
def estadisticas_resultados(ajustes, z_real=False):
    '''
    A partir de una lista de ajustes generada con la función
    "modelo_ajustado_y_resultados" entrega un DataFrame (sin 
    valores Nan) con los resultados del ajuste, que contiene:
    - el oid del objeto
    - ncall, chisq, ndof y el valor de chisq/ndof del mejor ajuste
    - los parámetros del mejor ajuste y los errores de los 
    parámetros ajustados (no se incluye la matriz de covarianza).
    
    Input:
    =================
    ajustes: [list], lista de ajustes del resultado de la función
            "modelo_ajustado_y_resultados".
    z_real: [bool], indica si en el ajuste se utilizó el redshift
            real de las SN.
    
    Output:
    =================
    df_resultados: [pd.DataFrame], DataFrame que en cada fila
                  contiene el oid, los resultados y mejores 
                  parámetros del ajuste de un cierto objeto.
                  Se eliminan aquellos que tienen valores Nan
                  en alguna de sus columnas
    '''
    oids = []
    ncall = []
    chisq = []
    ndof = [] # grados de libertad
    # nfit es siempre 1 ya que no se entrega el valor de la matriz
    #de covarianza, por lo que no se incluye en el df.
    z = []
    t0 = []
    x0 = []
    x1 = []
    c = []
    mwebv = []
    # mwr_v está fijo en 3.1
    # cov
    t0_err = []
    x0_err = []
    x1_err = []
    c_err = []

    for i in range(len(ajustes)):
        # oid 
        oid_i = ajustes[i][0]
        oids.append(oid_i)
        
        # estadísticas del ajuste
        ncall_i = ajustes[i][2]['ncall']
        ncall.append(ncall_i)
    
        chisq_i = ajustes[i][2]['chisq']
        chisq.append(chisq_i)
    
        ndof_i = ajustes[i][2]['ndof']
        ndof.append(ndof_i)

        z_i = ajustes[i][2]['parameters'][0]
        z.append(z_i)
    
        t0_i = ajustes[i][2]['parameters'][1]
        t0.append(t0_i)
    
        x0_i = ajustes[i][2]['parameters'][2]
        x0.append(x0_i)
    
        x1_i = ajustes[i][2]['parameters'][3]
        x1.append(x1_i)
    
        c_i = ajustes[i][2]['parameters'][4]
        c.append(c_i)
    
        mwebv_i = ajustes[i][2]['parameters'][5]
        mwebv.append(mwebv_i)
    
        # errores de los parámetros ajustados
        t0_err_i = ajustes[i][2]['errors']['t0']
        t0_err.append(t0_err_i)
    
        x0_err_i = ajustes[i][2]['errors']['x0']
        x0_err.append(x0_err_i)
    
        x1_err_i = ajustes[i][2]['errors']['x1']
        x1_err.append(x1_err_i)
    
        c_err_i = ajustes[i][2]['errors']['c']
        c_err.append(c_err_i)

    chisq_part_ndof = list(np.array(chisq)/np.array(ndof))
    
    if not z_real:
        # si no se utiliza el valor del redshift real de la SN
        # entonces este se ajusta, con lo que dentro del 
        # diccionario "errors" del resultado, se encuentra el
        # error del ajuste de z.
        z_err = []
        for i in range(len(ajustes)):
            z_err_i = ajustes[i][2]['errors']['z']
            z_err.append(z_err_i)
            
        data = {'oid': oids, 'ncall': ncall, 'chisq': chisq, 
                'ndof': ndof, 'chisq/ndof': chisq_part_ndof, 
                'z': z, 'z_err': z_err, 't0': t0, 't0_err': t0_err,
                'x0': x0, 'x0_err': x0_err, 'x1': x1, 
                'x1_err': x1_err, 'c': c, 'c_err': c_err, 
                'mwebv': mwebv}
    else:
        # si se utiliza el error del redshift real no hay error
        # en el ajuste de z, ya que este es un parámetro fijo
        data = {'oid': oids, 'ncall': ncall, 'chisq': chisq, 
                'ndof': ndof, 'chisq/ndof': chisq_part_ndof, 
                'z': z, 't0': t0, 't0_err': t0_err, 'x0': x0, 
                'x0_err': x0_err, 'x1': x1, 'x1_err': x1_err, 
                'c': c, 'c_err': c_err, 'mwebv': mwebv}  
        
    df_resultados = pd.DataFrame(data=data)
    
    # eliminar filas que tienen valores Nan
    return df_resultados.dropna()

In [28]:
def histogramas_estadisticas_resultados(ajustes, z_real=False, 
                                        n_bins=50, color='tab:blue',
                                        alpha=1, oids = np.array([])):
    '''
    A partir de una lista de ajustes generada con la función
    "modelo_ajustado_y_resultados" utiliza la función 
    "estadisticas_resultados" y crea histogramas para todas sus
    columnas (salvo la columna "oid"), imprimiendo el valor de su 
    rango, promedio, desviación estándar y mediana.
    
    Input:
    =================
    ajustes: [list], lista de ajustes del resultado de la función
            "modelo_ajustado_y_resultados".
            
    z_real: [bool], indica si en el ajuste se utilizó el redshift
            real de las SN.
            
    n_bins: [int], número de bins.
    
    color: [str], color del histograma.
    
    alpha: [float], transparencia del color.
    
    oids: [array of str], nombres de las SN de cuyas estadísticas se
          construye el histograma
    '''
    est = estadisticas_resultados(ajustes, z_real=z_real)
    
    if len(oids) == 0:
        df = estadisticas_resultados(ajustes, z_real=z_real)
        
    else:
        est = estadisticas_resultados(ajustes, z_real=z_real)
        df = est.loc[est['oid'].isin(oids)]
    
    for i in range(1, len(df.columns)):
        columna_i = df.columns[i]
        lista_i = df[columna_i]
        arreglo_i = np.array(lista_i)
        prom_i = np.mean(arreglo_i)
        std_i = np.std(arreglo_i)
        min_i = np.min(arreglo_i)
        max_i = np.max(arreglo_i)
        med_i = np.median(arreglo_i)
        
        # imprimir distribución de valores de la columna
        print('rango {} : {}, {}'.format(columna_i, min_i, max_i))
        print('promedio {} : {}'.format(columna_i, prom_i))
        print('desviación estándar {} : {}'.format(columna_i, 
                                                   std_i))
        print('mediana {} : {}'.format(columna_i, med_i))
        
        # construcción de un histograma 
        plt.figure(i, figsize=(6, 4))
        plt.hist(arreglo_i, bins= n_bins, color=color, alpha=alpha)[0]
        plt.title(columna_i , fontsize=14)
        plt.tick_params(labelsize=14)
        plt.show()

# Comparemos cómo se ven las estadísticas de los ajustes con $z$ fijo:

## - Corrección de la extinción de la vía láctea en los datos

### modelo SALT2

In [29]:
ajustes_SALT2_z_fijo_early_SNIa = modelo_ajustado_y_resultados(df_meta_early_SNIa, 
                                                               detect_early_SNIa, 
                                                               'salt2',
                                                               version='2.4', 
                                                               z_real=True)

/tmp/ipykernel_6426/252927044.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  detect_i['magpsf_SALT2'] = detect_i.apply(lambda x: x.magpsf - A_g if x.fid==1 else x.magpsf - A_r,
/tmp/ipykernel_6426/2311972296.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_detections_oid['f'] = df_detections_oid.apply(lambda x: flujo(x[mag_column]), axis=1)
/tmp/ipykernel_6426/2311972296.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer

### modelo SALT3

## - Corrección de la extinción de la vía láctea en el modelo

### modelo SALT2

### modelo SALT3

## - Sin corrección de la extinción de la vía láctea

### modelo SALT2

### modelo SALT3